# Chicago's Reported Crime Data Analysis in PySpark using Google Colab

## Download and install Spark

Check for files available files in the directory

In [2]:
!ls -l

total 4
drwxr-xr-x 1 root root 4096 Mar  3 01:40 sample_data


Update and install JDK

In [3]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:14 http://security.ubuntu.com/ubuntu focal-secur

Download Spark 3.2.2 and hadoop 3.2 

In [4]:
!wget -q http://archive.apache.org/dist/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz


See the list of files in the folder

In [5]:
!ls -l

total 294060
drwxr-xr-x 1 root root      4096 Mar  3 01:40 sample_data
-rw-r--r-- 1 root root 301112604 Jul 11  2022 spark-3.2.2-bin-hadoop3.2.tgz


Tar spark setup and install findspark library

In [6]:
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip install -q findspark

Set Java and Spark environment

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

## Setup environment

In [83]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

Initiate spark in-memory session

In [9]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

install pyspark library

In [10]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=e111f9966e2a79adb41351acb678dd9d1432aeacfc205c639aab6861bdbefd9c
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


## Downloading and preprocessing Chicago's Reported Crime Data

In [11]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD&bom=true&query=select+*

--2023-03-05 04:44:19--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.205, 52.206.68.26, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [ <=>                ]   1.70G  3.04MB/s    in 10m 18s 

2023-03-05 04:54:38 (2.82 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1828605180]



Check wether the file is downloaded

In [12]:
!ls -l

total 2079816
-rw-r--r--  1 root root 1828605180 Mar  4 12:36 'rows.csv?accessType=DOWNLOAD'
drwxr-xr-x  1 root root       4096 Mar  3 01:40  sample_data
drwxr-xr-x 13 1000 1000       4096 Jul 11  2022  spark-3.2.2-bin-hadoop3.2
-rw-r--r--  1 root root  301112604 Jul 11  2022  spark-3.2.2-bin-hadoop3.2.tgz


Rename the downloaded file to reported-crimes.csv

In [13]:
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv

In [84]:
!ls -l

total 2079848
-rw-r--r--  1 root root       5699 Aug 19  2019  police-station.csv
-rw-r--r--  1 root root 1828605180 Mar  4 12:36  reported-crimes.csv
-rw-r--r--  1 root root      22307 Dec  7  2021 'rows.csv?accessType=DOWNLOAD'
drwxr-xr-x  1 root root       4096 Mar  3 01:40  sample_data
drwxr-xr-x 13 1000 1000       4096 Jul 11  2022  spark-3.2.2-bin-hadoop3.2
-rw-r--r--  1 root root  301112604 Jul 11  2022  spark-3.2.2-bin-hadoop3.2.tgz


Filter the dataset and select the data with Date less than November 11th 2018

In [15]:
from pyspark.sql.functions import to_timestamp,col,lit

rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
#rc = spark.read.csv('reported-crimes.csv',header=True)
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

In [ ]:
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

In [16]:
rc.limit(5)

DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]

Retrive the fist five rows in the resultset

In [17]:
rc.head(5)


[Row(ID='10224738', Case Number='HY411648', Date=datetime.datetime(2015, 9, 5, 13, 30), Block='043XX S WOOD ST', IUCR='0486', Primary Type='BATTERY', Description='DOMESTIC BATTERY SIMPLE', Location Description='RESIDENCE', Arrest='false', Domestic='true', Beat='0924', District='009', Ward='12', Community Area='61', FBI Code='08B', X Coordinate='1165074', Y Coordinate='1875917', Year='2015', Updated On='02/10/2018 03:50:01 PM', Latitude='41.815117282', Longitude='-87.669999562', Location='(41.815117282, -87.669999562)'),
 Row(ID='10224739', Case Number='HY411615', Date=datetime.datetime(2015, 9, 4, 11, 30), Block='008XX N CENTRAL AVE', IUCR='0870', Primary Type='THEFT', Description='POCKET-PICKING', Location Description='CTA BUS', Arrest='false', Domestic='false', Beat='1511', District='015', Ward='29', Community Area='25', FBI Code='06', X Coordinate='1138875', Y Coordinate='1904869', Year='2015', Updated On='02/10/2018 03:50:01 PM', Latitude='41.895080471', Longitude='-87.765400451', 

##Schemas

Print the spark schema

In [19]:
rc.printSchema

<bound method DataFrame.printSchema of DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: string, Domestic: string, Beat: string, District: string, Ward: string, Community Area: string, FBI Code: string, X Coordinate: string, Y Coordinate: string, Year: string, Updated On: string, Latitude: string, Longitude: string, Location: string]>

In [85]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType


Get the list of columns in spark data frame

In [86]:
rc.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [ ]:
rc.show(5)

+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true|0924|     00

Create Spark schema for the dataframe

In [22]:
labels=[
    ('ID',StringType()),
    ('Case Number',StringType()),
    ('Date',TimestampType()),
 ('Block',StringType()),
 ('IUCR',StringType()),
 ('Primary Type', StringType()),
  ('Description',StringType()),
    ('Location Description',StringType()),
    ('Arrest',BooleanType()),
    ('Domestic',BooleanType()),
    ('Beat',StringType()),
    ('District',StringType()),
    ('Ward',IntegerType()),
    ('Community Area',IntegerType()),
    ('FBI Code',StringType()),
    ('X Coordinate',IntegerType()),
    ('Y Coordinate',IntegerType()),
    ('Year',IntegerType()),
    ('Updated On',TimestampType()),
    ('Latitude',DoubleType()),
    ('Longitude',DoubleType()),
    ('Location',StringType())
]

In [87]:
schema= StructType([StructField(x[0],x[1],True) for x in labels])

In [88]:
schema

StructType(List(StructField(ID,StringType,true),StructField(Case Number,StringType,true),StructField(Date,TimestampType,true),StructField(Block,StringType,true),StructField(IUCR,StringType,true),StructField(Primary Type,StringType,true),StructField(Description,StringType,true),StructField(Location Description,StringType,true),StructField(Arrest,BooleanType,true),StructField(Domestic,BooleanType,true),StructField(Beat,StringType,true),StructField(District,StringType,true),StructField(Ward,IntegerType,true),StructField(Community Area,IntegerType,true),StructField(FBI Code,StringType,true),StructField(X Coordinate,IntegerType,true),StructField(Y Coordinate,IntegerType,true),StructField(Year,IntegerType,true),StructField(Updated On,TimestampType,true),StructField(Latitude,DoubleType,true),StructField(Longitude,DoubleType,true),StructField(Location,StringType,true)))

Read the CSV file into spark dataframe having schema



In [25]:
rc=spark.read.csv('reported-crimes.csv',schema=schema)

In [26]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: timestamp (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [27]:
rc.show(5)

+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+
|      ID|Case Number|Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+
|      ID|Case Number|null|               Block|IUCR|Primary Type|         Description|Location Description|  null|    null|Beat|District|null|          null|FBI Code|        null|        null|null|      null|        null|    

## Working with Columns

Show first 5 values in IUCR column

In [28]:
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|IUCR|
|0486|
|0870|
|0810|
|2023|
+----+
only showing top 5 rows



## Display Columns


Select multiple columns in from the dataframe

In [30]:
rc.select('Case Number','Date','Block').show(5)

+-----------+----+--------------------+
|Case Number|Date|               Block|
+-----------+----+--------------------+
|Case Number|null|               Block|
|   HY411648|null|     043XX S WOOD ST|
|   HY411615|null| 008XX N CENTRAL AVE|
|   JC213529|null|082XX S INGLESIDE...|
|   HY411595|null|   035XX W BARRY AVE|
+-----------+----+--------------------+
only showing top 5 rows



## Add a column

In [89]:
from pyspark.sql.functions import lit

In [32]:
rc.withColumn('One',lit(1)).show(7)

+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+---+
|      ID|Case Number|Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|    Latitude|    Longitude|            Location|One|
+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+---+
|      ID|Case Number|null|               Block|IUCR|Primary Type|         Description|Location Description|  null|    null|Beat|District|null|          null|FBI Code|        null|        null|null|      null|     

In [33]:
rc.show(5)

+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+
|      ID|Case Number|Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+
|      ID|Case Number|null|               Block|IUCR|Primary Type|         Description|Location Description|  null|    null|Beat|District|null|          null|FBI Code|        null|        null|null|      null|        null|    

## Working with Rows

In [34]:
rc.show(5)

+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+
|      ID|Case Number|Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+----+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+------------+-------------+--------------------+
|      ID|Case Number|null|               Block|IUCR|Primary Type|         Description|Location Description|  null|    null|Beat|District|null|          null|FBI Code|        null|        null|null|      null|        null|    

In [35]:
twelth_nov = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') == lit('2018-11-12'))


In [36]:
twelth_nov.show(10)

+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|              Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11505149|   JB513151|2018-11-12 00:00:00| 003XX S WHIPPLE ST|0810|             THEFT|           OVER $500|              STREET| false| 

GroupBy on 'Primary Type'

In [38]:
rc.groupBy('Primary Type').count().orderBy('count',ascending=False).show(10)

+-------------------+-------+
|       Primary Type|  count|
+-------------------+-------+
|              THEFT|1634061|
|            BATTERY|1416503|
|    CRIMINAL DAMAGE| 882861|
|          NARCOTICS| 746436|
|            ASSAULT| 504006|
|      OTHER OFFENSE| 481129|
|           BURGLARY| 423368|
|MOTOR VEHICLE THEFT| 371195|
| DECEPTIVE PRACTICE| 341869|
|            ROBBERY| 291083|
+-------------------+-------+
only showing top 10 rows



In [39]:
rc.select('Arrest').distinct().show()

+------+
|Arrest|
+------+
|  null|
|  true|
| false|
+------+



In [40]:
rc.groupBy('Arrest').count().show()

+------+-------+
|Arrest|  count|
+------+-------+
|  null|      1|
|  true|2029533|
| false|5716105|
+------+-------+



In [41]:
rc.printSchema

<bound method DataFrame.printSchema of DataFrame[ID: string, Case Number: string, Date: timestamp, Block: string, IUCR: string, Primary Type: string, Description: string, Location Description: string, Arrest: boolean, Domestic: boolean, Beat: string, District: string, Ward: int, Community Area: int, FBI Code: string, X Coordinate: int, Y Coordinate: int, Year: int, Updated On: timestamp, Latitude: double, Longitude: double, Location: string]>

Percentage of Arrests happened throughout the dataset

In [42]:
rc.filter(col('Arrest')==True).count()/rc.count()

0.26202266849771855

Maximum number of crimes occured in which location

In [43]:
rc.groupBy('Location Description').count().orderBy('count',ascending=False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|2015739|
|           RESIDENCE|1300816|
|           APARTMENT| 871805|
+--------------------+-------+
only showing top 3 rows



##Builtin Function

In [44]:
from pyspark.sql import functions

In [45]:
print(dir(functions))

['Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_create_column_from_literal', '_create_lambda', '_create_udf', '_get_get_jvm_function', '_get_lambda_parameters', '_invoke_binary_math_function', '_invoke_function', '_invoke_function_over_column', '_invoke_higher_order_function', '_options_to_str', '_test', '_to_java_column', '_to_seq', '_unresolved_named_lambda_variable', 'abs', 'acos', 'acosh', 'add_months', 'aggregate', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'array_distinct', 'array_except', 'array_intersect', 'array_join', 'array_max', 'array_min', 'array_position', 'array_remove', 'array_repeat', 'array_sort', 'array_union', 'arrays_overlap', 'arrays_zip', 'asc', 'asc_nulls_first', 'asc_nulls_last', 'ascii', 'asin', 'asinh', 'assert_true', 'atan', 'atan2', 'atanh', 'a

## String Function

###Display the Primary Type column in lower and upper characters and the first four characters of the column


In [46]:
from pyspark.sql.functions import lower,upper,substring

In [47]:
help(substring)

Help on function substring in module pyspark.sql.functions:

substring(str, pos, len)
    Substring starts at `pos` and is of length `len` when str is String type or
    returns the slice of byte array that starts at `pos` in byte and is of length `len`
    when str is Binary type.
    
    .. versionadded:: 1.5.0
    
    Notes
    -----
    The position is not zero based, but 1 based index.
    
    Examples
    --------
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(substring(df.s, 1, 2).alias('s')).collect()
    [Row(s='ab')]



In [48]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: timestamp (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



String functions

In [49]:
rc.select(lower(col('Primary Type')),upper(col('Primary Type')),substring(col('Primary Type'),1,4)).show(5)

+-------------------+-------------------+-----------------------------+
|lower(Primary Type)|upper(Primary Type)|substring(Primary Type, 1, 4)|
+-------------------+-------------------+-----------------------------+
|       primary type|       PRIMARY TYPE|                         Prim|
|            battery|            BATTERY|                         BATT|
|              theft|              THEFT|                         THEF|
|              theft|              THEFT|                         THEF|
|          narcotics|          NARCOTICS|                         NARC|
+-------------------+-------------------+-----------------------------+
only showing top 5 rows



###Show the oldest and latest date

In [50]:
from pyspark.sql.functions import min,max

In [51]:
rc.select(min(col('Date')),max(col('Date'))).show(1)

+---------+---------+
|min(Date)|max(Date)|
+---------+---------+
|     null|     null|
+---------+---------+



In [52]:
from pyspark.sql.functions import to_date,to_timestamp,lit

In [53]:
df=spark.createDataFrame([('2019-12-25 13:30:00',),('2019-01-14 12:30:00',)],['Christmas'])
df.show()

+-------------------+
|          Christmas|
+-------------------+
|2019-12-25 13:30:00|
|2019-01-14 12:30:00|
+-------------------+



In [54]:
df.select(to_date(col('Christmas'),'yyyy-MM-dd HH:mm:ss'),to_timestamp(col('Christmas'),'yyyy-MM-dd HH:mm:ss')).show()

+---------------------------------------+--------------------------------------------+
|to_date(Christmas, yyyy-MM-dd HH:mm:ss)|to_timestamp(Christmas, yyyy-MM-dd HH:mm:ss)|
+---------------------------------------+--------------------------------------------+
|                             2019-12-25|                         2019-12-25 13:30:00|
|                             2019-01-14|                         2019-01-14 12:30:00|
+---------------------------------------+--------------------------------------------+



In [55]:
df=spark.createDataFrame([('25/Dec/2019 13:30:00',)],['Christmas'])

In [56]:
df

DataFrame[Christmas: string]

In [57]:
df.show()

+--------------------+
|           Christmas|
+--------------------+
|25/Dec/2019 13:30:00|
+--------------------+



In [58]:
df.select(to_date(col('Christmas'),'dd/MMM/yyyy HH:mm:ss'),to_timestamp(col('Christmas'),'dd/MMM/yyyy HH:mm:ss')).show(1)

+----------------------------------------+---------------------------------------------+
|to_date(Christmas, dd/MMM/yyyy HH:mm:ss)|to_timestamp(Christmas, dd/MMM/yyyy HH:mm:ss)|
+----------------------------------------+---------------------------------------------+
|                              2019-12-25|                          2019-12-25 13:30:00|
+----------------------------------------+---------------------------------------------+



In [59]:
df=spark.createDataFrame([('12/25/2019 01:30:00 PM',)],['Christmas'])

In [60]:
df.show(1,truncate=False)

+----------------------+
|Christmas             |
+----------------------+
|12/25/2019 01:30:00 PM|
+----------------------+



In [63]:
nrc=spark.read.csv('reported-crimes.csv',header=True)

In [64]:
nrc.show(5,truncate=False)

+--------+-----------+----------------------+---------------------+----+------------+-----------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+------------+-------------+-----------------------------+
|ID      |Case Number|Date                  |Block                |IUCR|Primary Type|Description            |Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On            |Latitude    |Longitude    |Location                     |
+--------+-----------+----------------------+---------------------+----+------------+-----------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------------------+------------+-------------+-----------------------------+
|10224738|HY411648   |09/05/2015 01:30:00 PM|043XX S WOOD ST      |0486|BATTERY     |DOMESTIC BAT

##Joins

Download Police-Station data

In [65]:
!wget https://data.cityofchicago.org/api/views/c7ck-438e/rows.csv?accessType=DOWNLOAD

--2023-03-05 05:09:41--  https://data.cityofchicago.org/api/views/c7ck-438e/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.205, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [ <=>                ]  21.78K  --.-KB/s    in 0s      

2023-03-05 05:09:42 (314 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [22307]



In [66]:
!wget https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD

--2023-03-05 05:09:59--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.68.26, 52.206.140.205, 52.206.140.199
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.68.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD.1’

rows.csv?accessType     [ <=>                ]   5.57K  --.-KB/s    in 0s      

2023-03-05 05:10:00 (1.04 GB/s) - ‘rows.csv?accessType=DOWNLOAD.1’ saved [5699]



In [67]:
!ls

 reported-crimes.csv		   sample_data
'rows.csv?accessType=DOWNLOAD'	   spark-3.2.2-bin-hadoop3.2
'rows.csv?accessType=DOWNLOAD.1'   spark-3.2.2-bin-hadoop3.2.tgz


In [68]:
!mv rows.csv\?accessType\=DOWNLOAD.1 police-station.csv 

In [69]:
!ll

/bin/bash: ll: command not found


In [70]:
ps=spark.read.csv('police-station.csv')

In [71]:
ps.show(5)

+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|         _c0|          _c1|                _c2|    _c3|  _c4|  _c5|                 _c6|         _c7|         _c8|         _c9|        _c10|        _c11|       _c12|        _c13|                _c14|
+------------+-------------+-------------------+-------+-----+-----+--------------------+------------+------------+------------+------------+------------+-----------+------------+--------------------+
|    DISTRICT|DISTRICT NAME|            ADDRESS|   CITY|STATE|  ZIP|             WEBSITE|       PHONE|         FAX|         TTY|X COORDINATE|Y COORDINATE|   LATITUDE|   LONGITUDE|            LOCATION|
|Headquarters| Headquarters|3510 S Michigan Ave|Chicago|   IL|60653|http://home.chica...|        null|        null|        null| 1177731.401| 1881697.404|41.83070169|-87.62339535|(41.8307016873, -

In [73]:
rc.show(1)

+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
| ID|Case Number|Date|Block|IUCR|Primary Type|Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|Latitude|Longitude|Location|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
| ID|Case Number|null|Block|IUCR|Primary Type|Description|Location Description|  null|    null|Beat|District|null|          null|FBI Code|        null|        null|null|      null|    null|     null|Location|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+

In [74]:
rc.select(col('Primary Type')).distinct().count()

37

In [75]:
rc.select(col('Primary Type')).distinct().orderBy(col('Primary Type')).show(35,truncate=False)

+---------------------------------+
|Primary Type                     |
+---------------------------------+
|ARSON                            |
|ASSAULT                          |
|BATTERY                          |
|BURGLARY                         |
|CONCEALED CARRY LICENSE VIOLATION|
|CRIM SEXUAL ASSAULT              |
|CRIMINAL DAMAGE                  |
|CRIMINAL SEXUAL ASSAULT          |
|CRIMINAL TRESPASS                |
|DECEPTIVE PRACTICE               |
|DOMESTIC VIOLENCE                |
|GAMBLING                         |
|HOMICIDE                         |
|HUMAN TRAFFICKING                |
|INTERFERENCE WITH PUBLIC OFFICER |
|INTIMIDATION                     |
|KIDNAPPING                       |
|LIQUOR LAW VIOLATION             |
|MOTOR VEHICLE THEFT              |
|NARCOTICS                        |
|NON - CRIMINAL                   |
|NON-CRIMINAL                     |
|NON-CRIMINAL (SUBJECT SPECIFIED) |
|OBSCENITY                        |
|OFFENSE INVOLVING CHILDREN 

Filter Non Criminal Data

In [76]:
nc=rc.filter((col('Primary Type')=='NON - CRIMINAL')|(col('Primary Type')=='NON-CRIMINAL')|(col('Primary Type')=='NON-CRIMINAL (SUBJECT SPECIFIED)'))

In [77]:
nc.count()

228

Find Top categories of cases of Non Criminal Data

In [78]:
nc.groupBy(col('Description')).count().orderBy('count',ascending=False).show(truncate=False)

+-------------------------------------------+-----+
|Description                                |count|
+-------------------------------------------+-----+
|LOST PASSPORT                              |111  |
|FOID - REVOCATION                          |76   |
|CONCEALED CARRY LICENSE REVOCATION         |17   |
|NOTIFICATION OF CIVIL NO CONTACT ORDER     |9    |
|NOTIFICATION OF STALKING - NO CONTACT ORDER|8    |
|FOUND PASSPORT                             |4    |
|GUN OFFENDER NOTIFICATION-NO CONTACT       |3    |
+-------------------------------------------+-----+



In [90]:
from pyspark.sql.functions import dayofweek

In [ ]:
help(dayofweek)

Help on function dayofweek in module pyspark.sql.functions:

dayofweek(col)
    Extract the day of the week of a given date as integer.
    
    .. versionadded:: 2.3.0
    
    Examples
    --------
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(dayofweek('dt').alias('day')).collect()
    [Row(day=4)]



In [80]:
rc.select(col('Date'),dayofweek(col('Date'))).show(4)

+----+---------------+
|Date|dayofweek(Date)|
+----+---------------+
|null|           null|
|null|           null|
|null|           null|
|null|           null|
+----+---------------+
only showing top 4 rows



In [81]:
from pyspark.sql.functions import date_format

In [82]:
rc.select(col('Date'),dayofweek(col('Date'))).show(4)

+----+---------------+
|Date|dayofweek(Date)|
+----+---------------+
|null|           null|
|null|           null|
|null|           null|
|null|           null|
+----+---------------+
only showing top 4 rows

